# **CNN Image & LSTM Text Classifier**
---
A deep learning project combining vision and natural languaging processing:
*   Convolutional Neural Network for image classification on the CIFAR-100 dataset, achieving ~37.5% test accuracy
*   Recurrent Neural Network for text classification on the Reuters dataset, achieving ~64% test accuracy
---
Fernanda :)

In [ ]:
# Importing all required libraries
import tensorflow as tf
import numpy as np
import random
from tensorflow import keras
from tensorflow.keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
from google.colab import drive

drive.mount('/content/drive')

In [ ]:
# Loading the data - training as well as testing
(train_data, test_data), ds_info = tfds.load(
    'cifar100',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

# Preparing the data that can be used by the next step - creating and training the DL model
def normalze_image(image, label):
  return tf.cast(image, tf.float32) / 255.0, label

train_data = train_data.map(normalze_image, num_parallel_calls=tf.data.AUTOTUNE)
train_data = train_data.cache().shuffle(1000).batch(64).prefetch(tf.data.AUTOTUNE)

test_data = test_data.map(normalze_image, num_parallel_calls=tf.data.AUTOTUNE)
test_data = test_data.batch(64).cache().prefetch(tf.data.AUTOTUNE)

# The data from TensforFlow and Keras. Each of those 100 integer class labels correspond to the following names, in the correct order
fine_labels = ['apple', 'aquarium_fish', 'baby', 'bear', 'beaver', 'bed', 'bee', 'beetle', 'bicycle', 'bottle', 'bowl', 'boy', 'bridge', 'bus', 'butterfly', 'camel', 'can', 'castle', 'caterpillar', 'cattle', 'chair', 'chimpanzee', 'clock', 'cloud', 'cockroach', 'couch', 'crab', 'crocodile', 'cup', 'dinosaur', 'dolphin', 'elephant', 'flatfish', 'forest', 'fox', 'girl', 'hamster', 'house', 'kangaroo', 'keyboard', 'lamp', 'lawn_mower', 'leopard', 'lion', 'lizard', 'lobster', 'man', 'maple_tree', 'motorcycle', 'mountain', 'mouse', 'mushroom', 'oak_tree', 'orange', 'orchid', 'otter', 'palm_tree', 'pear', 'pickup_truck', 'pine_tree', 'plain', 'plate', 'poppy', 'porcupine', 'possum', 'rabbit', 'raccoon', 'ray', 'road', 'rocket', 'rose', 'sea', 'seal', 'shark', 'shrew', 'skunk', 'skyscraper', 'snail', 'snake', 'spider', 'squirrel', 'streetcar', 'sunflower', 'sweet_pepper', 'table', 'tank', 'telephone', 'television', 'tiger', 'tractor', 'train', 'trout', 'tulip', 'turtle', 'wardrobe', 'whale', 'willow_tree', 'wolf', 'woman', 'worm']

# These are the string labels for the 20 superclasses
coarse_labels = ['aquatic_mammals', 'fish', 'flowers', 'food_containers', 'fruit_and_vegetables', 'household_electrical_devices', 'household_furniture', 'insects', 'large_carnivores', 'large_man-made_outdoor_things', 'large_natural_outdoor_scenes', 'large_omnivores_and_herbivores', 'medium_mammals', 'non-insect_invertebrates', 'people', 'reptiles', 'small_mammals', 'trees', 'vehicles_1', 'vehicles_2']


In [ ]:
# Visualizing the data by plotting 100 random images, one each for the 100 classes
# Drawing 10 images in one row, 10 rows total
fig = plt.figure(figsize=(15, 15))
label_image = {}
train_att = tfds.load('cifar100', split='train', as_supervised=True)

for image, label in tfds.as_numpy(train_att):
  if label not in label_image:
    label_image[label] = image
  if len(label_image) == 100:
    break

for idx, (label, image) in enumerate(label_image.items()):
  ax = fig.add_subplot(10, 10, idx+1)
  ax.imshow(image)
  ax.set_title(fine_labels[label], fontsize=7)
  ax.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
# Creating a DL model for Computer Vision - Convolutional Neural Network
cnn_modl = keras.models.Sequential([
    keras.Input(shape=(32, 32, 3)),
    keras.layers.Conv2D(32, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(128, (3, 3), activation='relu'),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(100, activation='softmax')
])

# Printing the DL model summary
cnn_modl.summary()

In [ ]:
# Setting the model checkpoints to be saved in a folder in the google drive at at location "/content/drive/My Drive/Colab Notebooks/cifar/"
checkpoint_path = "/content/drive/My Drive/Colab Notebooks/cifar/cp.weights.h5"
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                    save_weights_only=True,
                                                    save_best_only=True,
                                                    monitor='val_accuracy',
                                                    mode='max',
                                                    verbose=1)

In [ ]:
# Training the DL model using the training CIFAR-100 data
cnn_modl.compile(optimizer='adam',
                   loss='sparse_categorical_crossentropy',
                   metrics=['accuracy'])

hist = cnn_modl.fit(
    train_data,
    validation_data=test_data,
    epochs=20,
    callbacks=[checkpoint_cb]
)

In [ ]:
# Plotting the training/validation accuracy and loss
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)

plt.plot(hist.history['accuracy'], label='Training Accuracy', color='pink')
plt.plot(hist.history['val_accuracy'], label='Validation Accuracy', color='purple')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(hist.history['loss'], label='Training Loss', color='pink')
plt.plot(hist.history['val_loss'], label='Validation Loss', color='purple')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Re-initializing the model
reloaded_model = keras.Sequential([
    keras.Input(shape=(32, 32, 3)),
    keras.layers.Conv2D(32, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(128, (3, 3), activation='relu'),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(100, activation='softmax')
])

In [ ]:
# Loading the model weights that were saved at - "/content/drive/My Drive/Colab Notebooks/cifar/"
reloaded_model.load_weights("/content/drive/My Drive/Colab Notebooks/cifar/cp.weights.h5")

In [ ]:
# Evaluating the trained DL model on the CIFAR-100 test dataset
reloaded_model.compile(optimizer='adam',
                   loss='sparse_categorical_crossentropy',
                   metrics=['accuracy'])

test_loss, test_acc = reloaded_model.evaluate(test_data)
print(f"Test accuracy: {test_acc * 100: .2f}%")

In [ ]:
# Loading the Reuters dataset - using the Keras version
# Selecting the vocabulary size while loading the data
# The data will be loaded as integer representations for each word
vocab_size = 10000
(x_train, y_train), (x_test, y_test) = keras.datasets.reuters.load_data(num_words=vocab_size)

In [ ]:
# Preparing the data to be used for the next steps
# Each data entry (newswire) can be of different lengths
maxlen = 200
x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)

In [ ]:
# Creating a DL model for Natural Language Processing - Recurrent Neural Network / LSTM / GRU
rnn_model = keras.Sequential([
    keras.Input(shape=(maxlen,)),
    keras.layers.Embedding(input_dim=vocab_size, output_dim=64),
    keras.layers.LSTM(64),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(46, activation='softmax')
])

# Printing the DL model summary
rnn_model.summary()

In [ ]:
# Setting the model checkpoints to be saved in a folder in the google drive at at location "/content/drive/My Drive/Colab Notebooks/reuters/"
reuters_cp_path = "/content/drive/My Drive/Colab Notebooks/reuters/rnn_reuters.weights.h5"
reuters_cp_cb = tf.keras.callbacks.ModelCheckpoint(filepath=reuters_cp_path,
                                                    save_weights_only=True,
                                                    save_best_only=True,
                                                    monitor='val_accuracy',
                                                    mode='max',
                                                    verbose=1
)

In [ ]:
# Training the DL model using the training Reuters data
rnn_model.compile(optimizer='adam',
                   loss='sparse_categorical_crossentropy',
                   metrics=['accuracy'])
rnn_hist = rnn_model.fit(
    x_train,
    y_train,
    validation_data=(x_test, y_test),
    epochs=10,
    callbacks=[reuters_cp_cb]
)

In [ ]:
# Plotting the training and validation...accuracy and loss
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)

plt.plot(rnn_hist.history['accuracy'], label='Training Accuracy', color='pink')
plt.plot(rnn_hist.history['val_accuracy'], label='Validation Accuracy', color='purple')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(rnn_hist.history['loss'], label='Training Loss', color='pink')
plt.plot(rnn_hist.history['val_loss'], label='Validation Loss', color='purple')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Re-initializing the model
reloaded_rnn_model = keras.Sequential([
    keras.Input(shape=(maxlen,)),
    keras.layers.Embedding(input_dim=vocab_size, output_dim=64),
    keras.layers. LSTM(64),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(46, activation='softmax')
])

In [ ]:
# Loading the model weights that were saved at - "/content/drive/My Drive/Colab Notebooks/reuters/"
reloaded_rnn_model.load_weights(reuters_cp_path)

In [ ]:
# Evaluating the trained DL model on the Reuters test dataset
reloaded_rnn_model.compile(optimizer='adam',
                   loss='sparse_categorical_crossentropy',
                   metrics=['accuracy'])

test_loss, test_acc = reloaded_rnn_model.evaluate(x_test, y_test)
print(f"Test Accuracy: {test_acc * 100: .2f}%")